# Vectorisation des Nœuds pour GNN

Ce notebook transforme les labels textuels des nœuds en features numériques pour entraîner un GNN.

## Méthodes implémentées:
1. **Sentence Transformers** (RECOMMANDÉ) - Embeddings sémantiques
2. **TF-IDF + SVD** - Vectorisation classique
3. **Features hybrides** - Structure + sémantique

## 1. Chargement du dataset

In [ ]:
import pickle
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Charger le dataset
with open('mermaid_graphs.pkl', 'rb') as f:
    df = pickle.load(f)

print(f"✓ Dataset chargé: {len(df)} graphes")
print(f"Colonnes: {df.columns.tolist()}")
print(f"\nStatistiques:")
print(df[['num_nodes', 'num_edges']].describe())

## 2. Analyse des labels

In [ ]:
# Extraire tous les labels uniques
all_labels = []
for G in df['graph']:
    labels = [d.get('label', 'UNKNOWN') for n, d in G.nodes(data=True)]
    all_labels.extend(labels)

unique_labels = set(all_labels)

print(f"📊 Statistiques des labels:")
print(f"  - Total labels: {len(all_labels):,}")
print(f"  - Labels uniques: {len(unique_labels):,}")
print(f"  - Moyenne par graphe: {len(all_labels)/len(df):.1f}")
print(f"\n📝 Exemples de labels:")
print(list(unique_labels)[:20])

## 3. Méthode 1: Sentence Transformers (RECOMMANDÉ)

Utilise un modèle pré-entraîné pour créer des embeddings sémantiques de haute qualité.

**Avantages:**
- Capture la sémantique des concepts
- Gère les synonymes et variations
- Pas besoin de vocabulaire fixe
- Embeddings de dimension fixe (384 par défaut)

In [ ]:
# Installation si nécessaire
# !pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

# Charger le modèle (all-MiniLM-L6-v2: rapide et efficace, 384 dimensions)
print("🔄 Chargement du modèle Sentence Transformer...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print(f"✓ Modèle chargé: dimension d'embedding = {model.get_sentence_embedding_dimension()}")

# Créer un cache des embeddings pour éviter de recalculer
label_to_embedding = {}

print("\n🔄 Génération des embeddings pour tous les labels uniques...")
unique_labels_list = list(unique_labels)
embeddings = model.encode(unique_labels_list, show_progress_bar=True, batch_size=32)

# Créer le cache
for label, emb in zip(unique_labels_list, embeddings):
    label_to_embedding[label] = emb

print(f"\n✓ {len(label_to_embedding)} embeddings générés")
print(f"Dimension: {embeddings[0].shape[0]}")

In [ ]:
def add_sentence_embeddings_to_graph(G: nx.DiGraph, model, cache: dict) -> nx.DiGraph:
    """
    Ajoute les embeddings Sentence Transformer aux nœuds du graphe.
    """
    G_copy = G.copy()
    
    for node in G_copy.nodes():
        label = G_copy.nodes[node].get('label', 'UNKNOWN')
        
        # Utiliser le cache ou encoder si nouveau
        if label in cache:
            embedding = cache[label]
        else:
            embedding = model.encode([label])[0]
            cache[label] = embedding
        
        # Ajouter l'embedding comme feature
        G_copy.nodes[node]['features'] = embedding
    
    return G_copy

# Appliquer à tous les graphes
print("🔄 Application des embeddings à tous les graphes...")
tqdm.pandas(desc="Processing")
df['graph_with_embeddings'] = df['graph'].progress_apply(
    lambda g: add_sentence_embeddings_to_graph(g, model, label_to_embedding)
)

print("\n✓ Embeddings ajoutés à tous les graphes!")
print(f"\nExemple de nœud avec features:")
G_example = df['graph_with_embeddings'].iloc[0]
node_example = list(G_example.nodes(data=True))[0]
print(f"Node ID: {node_example[0]}")
print(f"Label: {node_example[1].get('label', 'N/A')}")
print(f"Features shape: {node_example[1]['features'].shape}")
print(f"Features preview: {node_example[1]['features'][:5]}...")

## 4. Méthode 2: TF-IDF + SVD

Approche classique de NLP pour vectoriser du texte.

**Avantages:**
- Rapide et léger
- Pas besoin de modèle externe
- Bon pour identifier les termes importants

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

# Créer le pipeline TF-IDF + SVD
print("🔄 Création du pipeline TF-IDF + SVD...")

tfidf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=1000,
        ngram_range=(1, 2),  # Unigrams et bigrams
        min_df=2,  # Apparaît au moins 2 fois
    )),
    ('svd', TruncatedSVD(
        n_components=128,  # Réduction à 128 dimensions
        random_state=42
    ))
])

# Fit sur tous les labels
print("Entraînement du pipeline sur tous les labels...")
tfidf_pipeline.fit(all_labels)

# Créer le cache
print("Génération des embeddings TF-IDF pour les labels uniques...")
label_to_tfidf = {}
for label in tqdm(unique_labels, desc="TF-IDF"):
    embedding = tfidf_pipeline.transform([label])[0]
    label_to_tfidf[label] = embedding

print(f"\n✓ {len(label_to_tfidf)} embeddings TF-IDF générés")
print(f"Dimension: {list(label_to_tfidf.values())[0].shape[0]}")

In [ ]:
def add_tfidf_embeddings_to_graph(G: nx.DiGraph, cache: dict, default_dim: int = 128) -> nx.DiGraph:
    """
    Ajoute les embeddings TF-IDF aux nœuds du graphe.
    """
    G_copy = G.copy()
    
    for node in G_copy.nodes():
        label = G_copy.nodes[node].get('label', 'UNKNOWN')
        
        # Utiliser le cache
        if label in cache:
            embedding = cache[label]
        else:
            # Fallback: vecteur zéro
            embedding = np.zeros(default_dim)
        
        G_copy.nodes[node]['features_tfidf'] = embedding
    
    return G_copy

# Appliquer à tous les graphes
print("🔄 Application des embeddings TF-IDF à tous les graphes...")
df['graph_with_tfidf'] = df['graph'].progress_apply(
    lambda g: add_tfidf_embeddings_to_graph(g, label_to_tfidf)
)

print("\n✓ Embeddings TF-IDF ajoutés!")

## 5. Méthode 3: Features Hybrides (Structure + Sémantique)

Combine les embeddings sémantiques avec des features structurelles du graphe.

**Features structurelles:**
- Degré du nœud (in/out/total)
- PageRank
- Betweenness centrality
- Clustering coefficient

In [ ]:
def compute_structural_features(G: nx.DiGraph) -> dict:
    """
    Calcule les features structurelles pour chaque nœud.
    Returns: {node_id: [feature_vector]}
    """
    features = {}
    
    # Gérer les graphes vides
    if G.number_of_nodes() == 0:
        return features
    
    # Calculer les métriques globales
    try:
        pagerank = nx.pagerank(G, max_iter=100)
    except:
        pagerank = {n: 1.0/max(1, G.number_of_nodes()) for n in G.nodes()}
    
    try:
        betweenness = nx.betweenness_centrality(G)
    except:
        betweenness = {n: 0.0 for n in G.nodes()}
    
    try:
        clustering = nx.clustering(G.to_undirected())
    except:
        clustering = {n: 0.0 for n in G.nodes()}
    
    # Pour chaque nœud, créer le vecteur de features
    for node in G.nodes():
        feature_vector = [
            G.in_degree(node),       # Degré entrant
            G.out_degree(node),      # Degré sortant
            G.degree(node),          # Degré total
            pagerank.get(node, 0.0),
            betweenness.get(node, 0.0),
            clustering.get(node, 0.0)
        ]
        features[node] = np.array(feature_vector, dtype=np.float32)
    
    return features

# Test sur un graphe non vide
non_empty_graphs = df[df['num_nodes'] > 0]['graph']
if len(non_empty_graphs) > 0:
    G_test = non_empty_graphs.iloc[0]
    struct_features = compute_structural_features(G_test)
    print(f"Features structurelles calculées pour {len(struct_features)} nœuds")
    print(f"Dimension: {list(struct_features.values())[0].shape[0]}")
    print(f"Exemple: {list(struct_features.values())[0]}")
else:
    print("⚠️ Aucun graphe non vide trouvé pour le test")

In [ ]:
from sklearn.preprocessing import StandardScaler

def add_hybrid_features(G: nx.DiGraph, embedding_cache: dict, model) -> nx.DiGraph:
    """
    Combine features structurelles + embeddings sémantiques.
    """
    G_copy = G.copy()
    
    # Gérer les graphes vides
    if G_copy.number_of_nodes() == 0:
        return G_copy
    
    # 1. Calculer features structurelles
    struct_features = compute_structural_features(G_copy)
    
    # 2. Normaliser les features structurelles
    struct_matrix = np.array(list(struct_features.values()))
    
    # Gérer le cas d'un seul nœud ou graphes très petits
    if struct_matrix.shape[0] <= 1:
        # Pas de normalisation possible, utiliser les features brutes
        struct_normalized = struct_matrix
    else:
        # Normalisation standard
        scaler = StandardScaler()
        struct_normalized = scaler.fit_transform(struct_matrix)
    
    # 3. Combiner avec embeddings
    for idx, node in enumerate(G_copy.nodes()):
        label = G_copy.nodes[node].get('label', 'UNKNOWN')
        
        # Embedding sémantique
        if label in embedding_cache:
            semantic_emb = embedding_cache[label]
        else:
            semantic_emb = model.encode([label])[0]
        
        # Features structurelles normalisées
        structural_emb = struct_normalized[idx]
        
        # Concaténer
        hybrid_features = np.concatenate([semantic_emb, structural_emb])
        
        G_copy.nodes[node]['features_hybrid'] = hybrid_features
    
    return G_copy

# Appliquer à tous les graphes
print("🔄 Création des features hybrides pour tous les graphes...")
df['graph_with_hybrid'] = df['graph'].progress_apply(
    lambda g: add_hybrid_features(g, label_to_embedding, model)
)

print("\n✓ Features hybrides créées!")
G_example = df['graph_with_hybrid'].iloc[0]
node_example = list(G_example.nodes(data=True))[0]
print(f"\nDimension features hybrides: {node_example[1]['features_hybrid'].shape[0]}")
print(f"  = {model.get_sentence_embedding_dimension()} (semantic) + 6 (structural)")

## 6. Visualisation des embeddings (t-SNE)

Visualiser les embeddings en 2D pour vérifier la qualité.

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Prendre un échantillon de labels
sample_labels = list(unique_labels)[:500]  # Limiter pour la visualisation
sample_embeddings = np.array([label_to_embedding[l] for l in sample_labels])

print("🔄 Réduction dimensionnelle avec t-SNE...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
embeddings_2d = tsne.fit_transform(sample_embeddings)

# Visualiser
plt.figure(figsize=(14, 10))
plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], alpha=0.6, s=50)

# Annoter quelques points
for i in range(0, len(sample_labels), 25):
    plt.annotate(sample_labels[i][:15], 
                (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                fontsize=8, alpha=0.7)

plt.title("Visualisation t-SNE des embeddings (Sentence Transformers)", fontsize=14)
plt.xlabel("t-SNE dimension 1")
plt.ylabel("t-SNE dimension 2")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✓ Les concepts similaires devraient être proches dans l'espace 2D")

## 7. Sauvegarder les résultats

Sauvegarder le DataFrame avec les 3 versions des graphes vectorisés.

In [ ]:
# Sauvegarder le DataFrame complet
with open('mermaid_graphs_vectorized.pkl', 'wb') as f:
    pickle.dump(df, f)

print(f"✓ Dataset sauvegardé: mermaid_graphs_vectorized.pkl")
print(f"  - {len(df)} graphes")
print(f"  - Colonnes: {df.columns.tolist()}")

# Sauvegarder aussi les caches d'embeddings
with open('embedding_caches.pkl', 'wb') as f:
    pickle.dump({
        'sentence_transformer': label_to_embedding,
        'tfidf': label_to_tfidf
    }, f)

print(f"\n✓ Caches d'embeddings sauvegardés: embedding_caches.pkl")

## 8. Résumé et recommandations

### 📊 Comparaison des méthodes

| Méthode | Dimension | Avantages | Inconvénients |
|---------|-----------|-----------|---------------|
| **Sentence Transformer** | 384 | ✅ Sémantique riche<br>✅ Robuste aux variations<br>✅ Pré-entraîné | ❌ Plus lourd<br>❌ Nécessite GPU (optionnel) |
| **TF-IDF + SVD** | 128 | ✅ Rapide<br>✅ Léger<br>✅ Pas de dépendances lourdes | ❌ Moins sémantique<br>❌ Sensible au vocabulaire |
| **Hybride** | 390 (384+6) | ✅ Meilleur des deux mondes<br>✅ Info structurelle | ❌ Plus de dimensions<br>❌ Plus lent |

### 🎯 Recommandation finale

Pour ton GNN, je recommande d'utiliser les **features hybrides** (`graph_with_hybrid`):

**Pourquoi ?**
1. Les **embeddings sémantiques** capturent le sens des concepts ("Student Centered", "Learning" sont proches)
2. Les **features structurelles** donnent des infos sur le rôle du nœud dans le graphe (hub, feuille, pont)
3. Le GNN peut apprendre à combiner ces deux types d'information

**Alternative:**
Si les graphes sont très petits (<10 nœuds), utiliser juste les **Sentence Transformers** peut suffire.

### 🚀 Prochaines étapes

1. Convertir les graphes NetworkX en format PyTorch Geometric
2. Créer des DataLoaders pour l'entraînement
3. Définir l'architecture du GNN (GCN, GAT, GraphSAGE)
4. Entraîner le modèle

In [ ]:
# Statistiques finales
print("\n" + "="*60)
print("📊 STATISTIQUES FINALES")
print("="*60)

print(f"\n✓ Dataset vectorisé avec succès!")
print(f"  - Nombre de graphes: {len(df)}")
print(f"  - Labels uniques: {len(unique_labels)}")
print(f"  - Total nœuds: {df['num_nodes'].sum():,}")
print(f"  - Total arêtes: {df['num_edges'].sum():,}")

print(f"\n📐 Dimensions des features:")
G_example = df['graph_with_embeddings'].iloc[0]
node_data = list(G_example.nodes(data=True))[0][1]
print(f"  - Sentence Transformer: {node_data['features'].shape[0]}")

G_example = df['graph_with_tfidf'].iloc[0]
node_data = list(G_example.nodes(data=True))[0][1]
print(f"  - TF-IDF + SVD: {node_data['features_tfidf'].shape[0]}")

G_example = df['graph_with_hybrid'].iloc[0]
node_data = list(G_example.nodes(data=True))[0][1]
print(f"  - Hybride: {node_data['features_hybrid'].shape[0]}")

print(f"\n💾 Fichiers générés:")
print(f"  - mermaid_graphs_vectorized.pkl (dataset complet)")
print(f"  - embedding_caches.pkl (caches pour réutilisation)")

print(f"\n🎯 Colonnes du DataFrame final:")
print(f"  {df.columns.tolist()}")